NERDS

[algospot의 NERDS 링크](https://algospot.com/judge/problem/read/NERDS)  
  
### 문제  
Algospot.com's annual programming contest is drawing near, and there are overwhelming number of registrations (10,000+!). We have only 5 judges, so we cannot handle this number of teams. Therefore, we decided to admit only the "true nerds" to the contest.  
  
JongMan came up with a theory that whether a person's nerdiness is dependent on a linear combination of two factors: their shoe size and their typing speed per minute. In his theory, the nerd score  
  
**F = A * (shoe size) + B * (typing speed per minute)**  
  
will be able to determine whether the guy is a nerd or not. The higher the score, the nerdier the guy is. We intend to set a threshold T; we will only admit people with scores equal to or above T.  
  
However, will this theory work? To find it out, we grabbed the data from some people we personally know. We know their shoe size and typing speeds, and we know whether each of them is a nerd or not. Will there be a set of parameters A, B and T which can successfully separate nerds from the others? Write a program to confirm this.
  
Please note that A, B and T can be arbitrary real numbers.  
  
### 출력  
The line must contain "THEORY HOLDS" if there are one or more set of parameters which satisfies the given data. "THEORY IS INVALID" should be printed in the other case. See the sample output for details.  

solution의  
  
nerds_info : 직사각형의 꼭지점의 좌표 [Nerd여부, 신발사이즈, 타이핑 속도]로 이루어진 리스트  
  
주어지는 정보의 수는 3이상 5000이하이다.
Nerd여부 -> 0: 너드가 아니다. 1: 너드이다.  
$(0 \leq 신발사이즈, 타이핑 속도 \leq 10000)$    
  

In [1]:
import math

def add(a,b): # 벡터의 합
    a_x, a_y = a
    b_x, b_y = b
    return (round(a_x+b_x,7), round(a_y+b_y,7))

def sub(a,b): # 벡터 뺄셈
    a_x, a_y = a
    b_x, b_y = b
    return (round(a_x-b_x,7), round(a_y-b_y,7))

def norm(a): # 벡터의 길이 구하기
    a_x, a_y = a
    return round(math.sqrt(pow(a_x,2)+pow(a_y,2)), 7)
    
def cross(a,b): # 벡터 외적
    a_x, a_y = a
    b_x, b_y = b
    return round((a_x*b_y) - (a_y*b_x),7)

def mul(multiple, a): # 상수를 벡터에 곱하기
    a_x, a_y = a
    return (round(multiple*a_x,7), round(multiple*a_y,7))

def min_vec(vertice): # 가장 왼쪽 아래에 위치한 점을 반환
    # y = -x + a 선을 기준으로 가장 아래에 위치하는 점이 작은 점.
    # x + y = a -> x,y의 합이 가장 작은 점이 작은 벡터
    x, y = vertice[0]
    for i in vertice[1:]:
        i_x, i_y = i
        if x+y == i_x+i_y:
            if x > i_x:
                x = i_x
                y = i_y
        elif x+y > i_x+i_y:
            x = i_x
            y = i_y
    return x, y

def segmentIntersects(a, b, c, d): # 두 선분이 접촉 or 교차하는지 여부 반환
    # 직선 ab 와 직선 cd에 대해서
    ab = cross(sub(b,a), sub(c,a)) * cross(sub(b,a), sub(d,a))
    # (ba, ac의 외적) * (ba, ad의 외적)
    cd = cross(sub(d,c), sub(a,c)) * cross(sub(d,c), sub(b,c))
    #(dc, ca의 외적) * (dc, cb의 외적)
    
    # 한쪽의 외적과 다른 한쪽의 외적값이 부호가 다를 것이다.
    # (+, -), (-, +), (0, 0), (0, +), (0, -) , (+, 0), (-, 0)
    # 그래서 (2개의 외적을 곱한 값 <=0 )일 경우 두 선분이 접촉 or 교차한다.
    if ab == 0 and cd == 0:
        if b < a:
            a,b = b,a
        if d < c:
            c,d = d,c
            return not(b < c or d < a);
    return ab <= 0 and cd <= 0

def isInside(q, poly): # 점 q가 다각형 poly내부에 존재하는지 여부 반환
    crosses = 0 # 점 q를 q의 y좌표를 기준으로 반직선을 그었을 때 다각형의 경계와 만나는 횟수
    q_x, q_y = q # 점 q의 좌표
    for i in range(len(poly)):
        p_i_x, p_i_y = poly[i] # 다각형에서의 한 꼭짓점 좌표
        j = (i+1) % len(poly)
        p_j_x, p_j_y = poly[j] # 인접한 꼭짓점의 좌표
        if (p_i_y > q_y) != (p_j_y > q_y): # p[i], p[j]가 반직선을 세로로 가르지를 때
            atX = (p_j_x - p_i_x) * (q_y - p_i_y) / (p_j_y - p_i_y) + p_i_x
            # 가로지르는 x좌표
            if q_x < atX: # q보다 오른쪽에서 가로지를 때
                crosses += 1 # 만나는 횟수 증가
    return (crosses % 2 > 0) # 만나는 횟수가 홀수일 때 poly내부에 점 q가 존재하는 것이다.

def giftWrap(vertice):
    # 꼭짓점들에 대해서 연결해서 다각형을 만들 때 가장 외부에서 다각형을 구성하는 꼭짓점 반환
    polygon = []
    min_point = min_vec(vertice) # 가장 왼쪽하단에 있는 꼭짓점
    polygon.append(min_point)
    while True:
        std_point = polygon[-1] # 기준점
        next_point = vertice[0] # 기준점에서 이어나갈 다음 꼭짓점
        for i in range(1, len(vertice)): 
            cross_val = cross(sub(next_point, std_point), sub(vertice[i], std_point))
            # 기존의 구성된 직선에 새로운 꼭짓점을 포함해서 외적을 구한다.
            # >0일 경우 : 현재까지 구한 다음 꼭짓점보다 외부에 있는 꼭짓점
            # =0일 경우 : 현재까지 예상된 다음 꼭짓점과 평행 선상
            dist = norm(sub(next_point, std_point)) - norm(sub(vertice[i], std_point))
            # 기준꼭짓점에서의 거리
            if cross_val > 0 or (cross_val == 0 and dist < 0):
                next_point = vertice[i]
                # 다음 꼭짓점을 갱신
        if next_point == min_point:
        # 만약 다음꼭짓점이 처음 시작한 최소 꼭짓점과 같을 경우
        # = 다각형 구성이 완성되었을 때
            break
        polygon.append(next_point)
    return polygon

def polygonIntersects(p1, p2): # 해당 다각형 p1과 p2의 겹치는 부분이 있는지 여부 반환
    if isInside(p1[0], p2) or isInside(p2[0], p1):
    # p1 ⊂ p2 이거나 p2 ⊂ p1일경우 참
        return True
    for i in range(len(p1)):
        for j in range(len(p2)):
            if segmentIntersects(p1[i], p1[(i+1)%len(p1)], p2[j], p2[(j+1)%len(p2)]):
            # p1의 구성하는 선분, p2 구성하는 선분끼리 교차할 경우 참
                return True
    return False # 그 외에는 거짓

def solution(nerds_info):
    # 신발 사이즈, 타이핑 속도를 각각 x,y 좌표로 가정한다.
    # 그래서 좌표평면상에 각각 구성하는 점을 이은 다각형을 만들경우
    # 교차할 경우 nerd인지 아닌지 여부를 판별할 수 없다.
    # 교차하지 않을 경우 nerd여부를 판별할 수 있다.
    
    p_nerd = [] # nerd를 구성하는 다각형
    p_n_nerd = [] # not-nerd를 구성하는 다각형
    for i in nerds_info:
        if i[0] == 1: # nerd 일 때
            p_nerd.append((i[1], i[2]))
        else: # nerd가 아닐 때
            p_n_nerd.append((i[1], i[2]))
            
    poly_nerd = giftWrap(p_nerd) # nerd 다각형
    poly_n_nerd = giftWrap(p_n_nerd) # not-nerd 다각형
    if polygonIntersects(poly_nerd, poly_n_nerd): # 2개의 다각형이 교차할 경우
        return "THEORY IS INVALID" # nerd 여부 판별 불가
    return "THEORY HOLDS" # 교차하지 않는다면 nerd여부를 판별할 수 있다.



In [2]:
nerds_info = [(1, 2, 3), 
              (1, 3, 4),
              (1, 4, 5),
              (1, 2, 5),
              (0, 4, 1),
              (0, 5, 5),
              (0, 3, 3),
              (0, 4, 4)]

In [3]:
solution(nerds_info)

'THEORY HOLDS'

In [4]:
nerds_info = [(1, 1, 5),
              (1, 5, 1),
              (1, 1, 1),
              (0, 2, 2),
              (0, 4, 1),
              (0, 1, 4)]

In [5]:
solution(nerds_info)

'THEORY IS INVALID'

In [6]:
nerds_info = [(1, 10, 10),
              (0, 10, 10),
              (1, 5, 15),
              (1, 5, 5),
              (0, 15, 15),
              (0, 15, 5)]

In [7]:
solution(nerds_info)

'THEORY IS INVALID'